In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import math

from datetime import datetime, timedelta

from typing import Tuple

from tqdm import tqdm

import random

In [2]:
df = pd.read_csv('connection_graph.csv')
df = df.drop( columns=["Unnamed: 0.1","Unnamed: 0","company"])

C:\Users\romaf\AppData\Local\Temp\ipykernel_49164\1888819752.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('connection_graph.csv')


In [3]:
class Node:

    def __init__(self, name,x,y) -> None:
        self.name = name
        self.x = x
        self.y = y
        self.connections = set()

    def __repr__(self) -> str:
        return ( f"Station({self.name})")
    

    def get_closest_connections(self, at_time,lines_important=True):
        out_list = dict()

        if lines_important:
            for possible_conn in self.connections:
                if possible_conn[2] >= at_time:
                    if (possible_conn[0],possible_conn[1]) in out_list:

                        if out_list[(possible_conn[0],possible_conn[1])][2] > possible_conn[2]:
                            out_list[(possible_conn[0],possible_conn[1])] = possible_conn
                    else:
                        out_list[(possible_conn[0],possible_conn[1])] = possible_conn
        else:
             for possible_conn in self.connections:
                if possible_conn[2] >= at_time:
                    if (possible_conn[0]) in out_list:

                        if out_list[possible_conn[0]][2] > possible_conn[2]:
                            out_list[possible_conn[0]] = possible_conn
                    else:
                        out_list[possible_conn[0]] = possible_conn
            
        return set( out_list.values())
    
class Graph:

    def __init__(self) -> None:

        self.nodes = dict()


    def prep_path(self,path) -> str:

        return "\n".join( [ ' To {} on line {} at {} -> Arrival at {}'.format(x[0],x[1],x[2].strftime('%H:%M:%S'),x[3].strftime('%H:%M:%S')) for x in path])


    def nodes_distance(self,node1:Node,node2:Node):
        return math.sqrt(math.pow(node1.x-node2.x,2) + math.pow(node1.y-node2.y,2))


    def load_data(self,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon) -> None:
        
        if start_stop not in self.nodes:
            self.nodes[start_stop] = Node(start_stop,start_stop_lat,start_stop_lon)

        if end_stop not in self.nodes:
            self.nodes[end_stop] = Node(end_stop,end_stop_lat,end_stop_lon)

        self.nodes[start_stop].connections.add((self.nodes[end_stop],line,datetime.strptime(departure_time,"%H:%M:%S"),datetime.strptime(arrival_time,"%H:%M:%S")))

    def eval_route(self,start_time,route,time_criterion=False):

        if time_criterion:

            total_time = timedelta(seconds=0)
            total_inst = ''
            curr_time = start_time
            for ind in range(1, len(route)):
                res = self.astar(start=route[ind-1], end=route[ind],start_time=curr_time,time_criterion=True)
                if res is None:
                    return timedelta(days=9999),""
                time_cost,arrival_time,instr  =res
                total_time += time_cost
                total_inst += instr
                curr_time = arrival_time
            return total_time,total_inst

        else:
            
            total_transfers = 0
            total_inst = ''

            curr_time = start_time
            for ind in range(1, len(route)):
                res = self.astar(start=route[ind-1], end=route[ind],start_time=curr_time,time_criterion=False)
                if res is None:
                    return 9999999,""
                transfers,arrival_time,instr =res
                total_transfers += transfers
                total_inst += instr
                curr_time = arrival_time
            return total_transfers,total_inst
            


    def TabuSearch(self,start, list_of_stops, start_time,time_criterion=False, iters = 10, random_neighborhood_gens = False):

        tabu_size = len(list_of_stops) + 2 // 4
        # Origin : https://www.researchgate.net/publication/220471567_Optimizing_tabu_list_size_for_the_traveling_salesman_problem
        # Taken from:  
        # https://www.researchgate.net/publication/368159076_Multi-Tour_Set_Traveling_Salesman_Problem_in_Planning_Power_Transmission_Line_Inspection
        # they used this ratio, seems good 

        def swap_stations(route,station1, station2):
            out_route = route.copy()
            out_route[station1],out_route[station2] = out_route[station2],out_route[station1]
            return out_route
        
        # Source https://www.researchgate.net/publication/255599657_Analysis_of_neighborhood_generation_and_move_selection_strategies_on_the_performance_of_Tabu_Search
        def gen_neighborhood_routes_pairwise_interchange(route):

            out_routes = []

            for i in range(1, len(route) - 1):
                for j in range(1, len(route) - 1):
                    if i == j:
                        continue

                    out_routes.append(swap_stations(route,i,j))

            return out_routes
        
        # Source https://www.researchgate.net/publication/255599657_Analysis_of_neighborhood_generation_and_move_selection_strategies_on_the_performance_of_Tabu_Search
        def gen_neighborhood_routes_random_neighborhood(route):

            out_routes = []

            numbers_size = random.randint( 1, len(route) - 1 )

            for _ in range(numbers_size):

                curr_pos =  random.randint( 1, len(route) - 1 )

                next_pos = random.randint( 1, len(route) - 1 )

                if curr_pos != next_pos:
                    out_routes.append(swap_stations(route,curr_pos,next_pos))

            return out_routes


        best_route = [start,*list_of_stops,start]
        best_route_eval,best_route_inst= self.eval_route(start_time,best_route,time_criterion)
        best_in_hood = best_route

        tabu = []

        for _ in tqdm(range(iters)):


            if random_neighborhood_gens:
                hood = gen_neighborhood_routes_random_neighborhood(best_in_hood)
            else:

                hood = gen_neighborhood_routes_pairwise_interchange(best_in_hood)

            best_in_hood = hood[0]
            best_in_hood_eval,best_in_hood_inst = self.eval_route(start_time,best_in_hood,time_criterion)

            for cand_route in hood: 

                cand_eval,cand_inst = self.eval_route(start_time,cand_route,time_criterion)
                if  (cand_eval < best_in_hood_eval) and cand_route not in tabu:

                    best_in_hood = cand_route
                    best_in_hood_eval = cand_eval
                    best_in_hood_inst = cand_inst
                
                # Aspiration criterion  
                if cand_route in tabu and cand_eval < best_route_eval:
                    best_in_hood = cand_route
                    best_in_hood_eval = cand_eval
                    best_in_hood_inst = cand_inst


            if best_in_hood_eval < best_route_eval:
                best_route = best_in_hood
                best_route_eval = best_in_hood_eval
                best_route_inst = best_in_hood_inst

            tabu.append(best_in_hood)

            if len(tabu) > tabu_size:
                tabu.pop(0)

        return best_route,best_route_inst
        



    def astar(self,start,end, start_time, time_criterion=False,):

        curr_time = datetime.strptime(start_time,"%H:%M:%S")

        if time_criterion:
             
            target_not_found = True

            nodes_distances = dict()
            nodes_time = dict()
            nodes_connections_path = dict()

            visited = set()

            visited.add(start)
            nodes_time[start] = 0
            nodes_connections_path[start] = []

            for key,node in self.nodes.items():
                nodes_distances[key] = self.nodes_distance(node,self.nodes[end])

            for first_nodes in self.nodes[start].get_closest_connections(curr_time,False):

                if start == first_nodes[0].name : continue

                time_to_visit = (first_nodes[3] - curr_time).seconds

                nodes_time[first_nodes[0].name] = time_to_visit

                nodes_connections_path[first_nodes[0].name] = nodes_connections_path[start].copy()
                nodes_connections_path[first_nodes[0].name].append(first_nodes)
            

            while target_not_found:

                next_node = None
                next_node_score = None

                for key in nodes_time.keys():
                    if key not in visited:
                        next_node_candidate_score = nodes_distances[key]*1000 + nodes_time[key]
                        if next_node is None or next_node_score > next_node_candidate_score:
                            next_node = key
                            next_node_score = next_node_candidate_score
                
                visited.add(next_node)
                
                if next_node is None:
                    return None

                for explored_nodes_con in self.nodes[next_node].get_closest_connections(curr_time + timedelta(seconds=nodes_time[next_node]),False):

                    if next_node == explored_nodes_con[0].name: continue

                    nodes_time[explored_nodes_con[0].name] = (explored_nodes_con[3] - curr_time).seconds

                    nodes_connections_path[explored_nodes_con[0].name] = nodes_connections_path[next_node].copy()
                    nodes_connections_path[explored_nodes_con[0].name].append(explored_nodes_con)

                    if explored_nodes_con[0].name == end:
                        out="\n"
                        out+= f'\nFrom {start} '
                        out+= self.prep_path(nodes_connections_path[end])
                        out+="\nTime taken : {}".format(timedelta(seconds=nodes_time[explored_nodes_con[0].name]))
                        out+= "\nArrival at : {} ".format( (curr_time + timedelta(seconds=nodes_time[explored_nodes_con[0].name])).strftime("%H:%M:%S") )
                        return (timedelta(seconds=nodes_time[explored_nodes_con[0].name]),(curr_time + timedelta(seconds=nodes_time[explored_nodes_con[0].name])).strftime("%H:%M:%S"),out)

        else:
            # node line dep arr

            def explore_line(node,depth,target_node,time,line,visited_list):

                if depth >=2:
                    for conns in node.get_closest_connections(time,lines_important=True):
                        res = explore_line(conns[0],depth-1,target_node,conns[3],conns[1],[node])
                        if res is not None:
                            resc  = res.copy()
                            resc.append(conns)
                            return resc

                if depth == 1:
                    for conns in node.get_closest_connections(time,lines_important=True):
                        if conns[0] in visited_list:
                            continue
                        if conns[1] == line:
                            if conns[0].name == target_node:
                                return [conns]
                            vl = visited_list.copy()
                            vl.append(node)
                            res = explore_line(conns[0],1,target_node,conns[3],line,vl)
                            if res is None:
                                return res
                            else:
                                resc  = res.copy()
                                resc.append(conns)
                                return resc
                    return None
           
            res = None
            for lines_limit in range(1,4):
                for cons in self.nodes[start].get_closest_connections(curr_time,lines_important=True):
                    res = explore_line(self.nodes[start],lines_limit,end,curr_time,cons[1],[])
                    if res is not None:
                        break
                if res is not None:
                        break
            if res is not None:
                res.reverse()
                out="\n"
                out+= f'\nFrom {start} '
                out+= self.prep_path(res)
                out+="\nTime taken : {}".format(res[-1][3] - curr_time)
                out+= "\nArrival at : {} ".format(res[-1][3].strftime("%H:%M:%S"))
                return (lines_limit,res[-1][3].strftime("%H:%M:%S") ,out)              
            else:
                return None
            

In [4]:
all_stops = df['end_stop'].unique()

test = []

for _ in range(10):
    pack = []
    for _ in range(random.randint(3,6)):
        x = random.randint(0, len(all_stops))
        pack.append(all_stops[x])
    test.append(pack)
test

[['Różanka', 'Bajana', 'Czerna'],
 ['Wysoka - Chabrowa', 'Kowieńska', 'KŁOKOCZYCE'],
 ['Morwowa', 'Raków III', 'Zacisze', 'Sąsiedzka', 'Żerniki'],
 ['Smolec - Wiśniowa', 'Wrocławski Park Technologiczny', 'DWORZEC AUTOBUSOWY'],
 ['Szczepin',
  'ZWYCIĘSKA',
  'Bukowina',
  'Pruszowice - las',
  'Szkocka',
  'Raków I'],
 ['Ojca Beyzyma', 'Strachowicka', 'Domaszczyn - Wrocławska sklep'],
 ['ŚWINIARY',
  'Kamieńskiego',
  'Kopańskiego',
  'Pasikurowice - skrzy. Malinowa',
  'Mickiewicza'],
 ['Hala Targowa',
  'Krzeptów - pętla',
  'ROD Storczyk',
  'Buforowa-Rondo',
  'Kosmonautów (Szpital)',
  'Połabian'],
 ['Maślicka (Staw)', 'Inżynierska', 'Mrozów - Świetlica', 'Orzechowa'],
 ['Smolec - Główna (na wys. nr 52)',
  'Małkowice - Główna',
  'Długołęka - SELGROS pętla',
  'Jastrzębia',
  'Piramowicza',
  'Piwnika-Ponurego']]

In [5]:
gr = Graph()

for ind,row in tqdm(df.iterrows(),total=273471):
    gr.load_data(*row.values.flatten().tolist())

100%|██████████| 273471/273471 [00:39<00:00, 6991.12it/s]


### To fix 

In [6]:
gr.astar('Reja','Kleczkowska','17:59:00',False)

(3,
 '21:21:00',
 '\n\nFrom Reja  To Station(PL. GRUNWALDZKI) on line 2 at 18:10:00 -> Arrival at 18:13:00\n To Station(most Grunwaldzki) on line D at 18:17:00 -> Arrival at 18:19:00\n To Station(Poczta Główna) on line D at 18:19:00 -> Arrival at 18:21:00\n To Station(GALERIA DOMINIKAŃSKA) on line D at 18:21:00 -> Arrival at 18:24:00\n To Station(Renoma) on line D at 18:24:00 -> Arrival at 18:28:00\n To Station(Arkady (Capitol)) on line D at 18:28:00 -> Arrival at 18:30:00\n To Station(Świdnicka) on line D at 21:10:00 -> Arrival at 21:12:00\n To Station(Rynek) on line D at 21:12:00 -> Arrival at 21:14:00\n To Station(Mosty Pomorskie) on line D at 21:14:00 -> Arrival at 21:15:00\n To Station(Pomorska) on line D at 21:15:00 -> Arrival at 21:16:00\n To Station(Pomorska) on line D at 21:16:00 -> Arrival at 21:17:00\n To Station(pl. Strzelecki) on line D at 21:17:00 -> Arrival at 21:19:00\n To Station(Kleczkowska) on line D at 21:19:00 -> Arrival at 21:21:00\nTime taken : 3:22:00\nArrival a

In [52]:
for test_case in test:
    route, path = gr.TabuSearch(test_case[0],test_case[1:], '10:00:00',True,iters=5)
    print(route)
    for line in path.split('\n'):
        print(line)

100%|██████████| 5/5 [01:02<00:00, 12.57s/it]


['Kowieńska', 'Zajezdnia GAJ', 'Miękinia - pętla', 'Źródła', 'Małkowice - Główna', 'Kiełczów - Boczna', 'Kowieńska']


From Kowieńska  To Station(Żmudzka) on line 121 at 10:04:00 -> Arrival at 10:05:00
 To Station(Kiełczowska) on line 131 at 10:07:00 -> Arrival at 10:09:00
 To Station(Poleska) on line 131 at 10:09:00 -> Arrival at 10:10:00
 To Station(KIEŁCZOWSKA (LZN)) on line 131 at 10:10:00 -> Arrival at 10:11:00
 To Station(Psie Pole (Rondo Lotników Polskich)) on line 131 at 10:11:00 -> Arrival at 10:13:00
 To Station(Psie Pole) on line 128 at 10:13:00 -> Arrival at 10:14:00
 To Station(Zielna) on line 128 at 10:14:00 -> Arrival at 10:15:00
 To Station(C.H. Korona) on line 128 at 10:15:00 -> Arrival at 10:17:00
 To Station(Brücknera) on line 121 at 10:17:00 -> Arrival at 10:19:00
 To Station(Kwidzyńska) on line 121 at 10:19:00 -> Arrival at 10:21:00
 To Station(Zacisze) on line 121 at 10:21:00 -> Arrival at 10:22:00
 To Station(Śniadeckich) on line 121 at 10:22:00 -> Arrival at 10:

100%|██████████| 5/5 [00:34<00:00,  6.88s/it]


['Długołęka - Nowy Urząd', 'Dobroszów - skrzy. Węgierska', 'POLANOWICE', 'Broniewskiego', 'Wapienna', 'Długołęka - Nowy Urząd']


From Długołęka - Nowy Urząd  To Station(Długołęka - Kościół) on line 941 at 10:59:00 -> Arrival at 11:00:00
 To Station(Długołęka - Broniewskiego (na wys. nr 11)) on line 941 at 11:00:00 -> Arrival at 11:01:00
 To Station(Długołęka - Parkowa/skrzy.) on line 941 at 11:01:00 -> Arrival at 11:04:00
 To Station(Szczodre - Trzebnicka (na wys. nr 3a)) on line 941 at 11:04:00 -> Arrival at 11:05:00
 To Station(Szczodre - Trzebnicka) on line 941 at 11:05:00 -> Arrival at 11:06:00
 To Station(Szczodre - pętla) on line 941 at 11:06:00 -> Arrival at 11:08:00
 To Station(Szczodre - Szkoła) on line 904 at 11:47:00 -> Arrival at 11:48:00
 To Station(Szczodre - stawy) on line 904 at 11:48:00 -> Arrival at 11:50:00
 To Station(Łosice - plac zabaw) on line 904 at 11:50:00 -> Arrival at 11:52:00
 To Station(Budziwojowice) on line 904 at 11:52:00 -> Arrival at 11:54:00
 To Sta

100%|██████████| 5/5 [00:08<00:00,  1.72s/it]


...

100%|██████████| 5/5 [00:06<00:00,  1.26s/it]


['Szkocka', 'Małopanewska', 'Szczodre - Szkoła', 'Szkocka']


From Szkocka  To Station(Gądowianka) on line 136 at 10:05:00 -> Arrival at 10:07:00
 To Station(Na Ostatnim Groszu) on line 136 at 10:07:00 -> Arrival at 10:09:00
 To Station(Kwiska) on line 143 at 10:09:00 -> Arrival at 10:12:00
 To Station(Małopanewska) on line 33 at 10:12:00 -> Arrival at 10:13:00
Time taken : 0:13:00
Arrival at : 10:13:00 

From Małopanewska  To Station(Niedźwiedzia) on line 33 at 10:13:00 -> Arrival at 10:15:00
 To Station(ZACHODNIA (Stacja kolejowa)) on line 33 at 10:15:00 -> Arrival at 10:18:00
 To Station(pl. Strzegomski (Muzeum Współczesne)) on line 33 at 10:18:00 -> Arrival at 10:20:00
 To Station(Młodych Techników) on line 33 at 10:20:00 -> Arrival at 10:21:00
 To Station(PL. JANA PAWŁA II) on line 33 at 10:21:00 -> Arrival at 10:23:00
 To Station(Rynek) on line 33 at 10:23:00 -> Arrival at 10:25:00
 To Station(Zamkowa) on line 33 at 10:25:00 -> Arrival at 10:26:00
 To Station(Świdnicka) on line 3

 40%|████      | 2/5 [00:33<00:50, 16.75s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "C:\Users\romaf\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\romaf\AppData\Local\Temp\ipykernel_46588\3561158504.py", line 2, in <module>
    route, path = gr.TabuSearch(test_case[0],test_case[1:], '10:00:00',True,iters=5)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\romaf\AppData\Local\Temp\ipykernel_46588\2441840908.py", line 164, in TabuSearch
    cand_eval,cand_inst = self.eval_route(start_time,cand_route,time_criterion)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\romaf\AppData\Local\Temp\ipykernel_46588\2441840908.py", line 71, in eval_route
    res = self.astar(start=route[ind-1], end=route[ind],start_time=curr_time,time_criterion=True)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [18]:
route, path =  gr.TabuSearch('KRZYKI',['Reja','Inżynierska','Katedra','pl. Orląt Lwowskich'],'13:00:00',True,iters=20,random_neighborhood_gens=False)

100%|██████████| 20/20 [00:25<00:00,  1.29s/it]


In [19]:
print(route)
for line in path.split('\n'):
    print(line)

['KRZYKI', 'Inżynierska', 'Reja', 'Katedra', 'pl. Orląt Lwowskich', 'KRZYKI']


From KRZYKI  To Station(Orla) on line 6 at 13:00:00 -> Arrival at 13:02:00
 To Station(Jastrzębia) on line 6 at 13:02:00 -> Arrival at 13:03:00
 To Station(Hallera) on line 6 at 13:03:00 -> Arrival at 13:04:00
 To Station(Gajowicka) on line 136 at 13:05:00 -> Arrival at 13:07:00
 To Station(Mielecka) on line 136 at 13:07:00 -> Arrival at 13:08:00
 To Station(Ojca Beyzyma) on line 136 at 13:08:00 -> Arrival at 13:10:00
 To Station(Aleja Pracy) on line 136 at 13:10:00 -> Arrival at 13:12:00
 To Station(Inżynierska) on line A at 13:14:00 -> Arrival at 13:16:00
Time taken : 0:16:00
Arrival at : 13:16:00 

From Inżynierska  To Station(Krucza (Mielecka)) on line A at 13:16:00 -> Arrival at 13:18:00
 To Station(Krucza) on line A at 13:18:00 -> Arrival at 13:20:00
 To Station(Pl. Hirszfelda) on line A at 13:20:00 -> Arrival at 13:23:00
 To Station(Arkady (Capitol)) on line A at 13:23:00 -> Arrival at 13:30:00
 To S

In [17]:
print(route)
for line in path.split('\n'):
    print(line)

['KRZYKI', 'Inżynierska', 'Reja', 'Katedra', 'pl. Orląt Lwowskich', 'KRZYKI']


From KRZYKI  To Station(Orla) on line 6 at 13:00:00 -> Arrival at 13:02:00
 To Station(Jastrzębia) on line 6 at 13:02:00 -> Arrival at 13:03:00
 To Station(Hallera) on line 6 at 13:03:00 -> Arrival at 13:04:00
 To Station(Gajowicka) on line 136 at 13:05:00 -> Arrival at 13:07:00
 To Station(Mielecka) on line 136 at 13:07:00 -> Arrival at 13:08:00
 To Station(Ojca Beyzyma) on line 136 at 13:08:00 -> Arrival at 13:10:00
 To Station(Aleja Pracy) on line 136 at 13:10:00 -> Arrival at 13:12:00
 To Station(Inżynierska) on line A at 13:14:00 -> Arrival at 13:16:00
Time taken : 0:16:00
Arrival at : 13:16:00 

From Inżynierska  To Station(Krucza (Mielecka)) on line A at 13:16:00 -> Arrival at 13:18:00
 To Station(Krucza) on line A at 13:18:00 -> Arrival at 13:20:00
 To Station(Pl. Hirszfelda) on line A at 13:20:00 -> Arrival at 13:23:00
 To Station(Arkady (Capitol)) on line A at 13:23:00 -> Arrival at 13:30:00
 To S

In [46]:
route, path =  gr.TabuSearch('KRZYKI',['Reja','Inżynierska','Katedra','pl. Orląt Lwowskich'],'13:00:00',False)

100%|██████████| 10/10 [36:01<00:00, 216.10s/it]


In [47]:
print(route)
for line in path.split('\n'):
    print(line)

['KRZYKI', 'Inżynierska', 'pl. Orląt Lwowskich', 'Katedra', 'Reja', 'KRZYKI']


From KRZYKI  To Station(Sowia) on line A at 13:12:00 -> Arrival at 13:13:00
 To Station(Chłodna) on line A at 13:13:00 -> Arrival at 13:14:00
 To Station(Wawrzyniaka) on line A at 13:14:00 -> Arrival at 13:15:00
 To Station(Rymarska) on line A at 13:15:00 -> Arrival at 13:17:00
 To Station(RACŁAWICKA) on line A at 13:17:00 -> Arrival at 13:18:00
 To Station(Bukowskiego) on line A at 13:18:00 -> Arrival at 13:19:00
 To Station(Stanki) on line A at 13:19:00 -> Arrival at 13:20:00
 To Station(Kadłubka) on line A at 13:20:00 -> Arrival at 13:21:00
 To Station(Wiejska) on line A at 13:21:00 -> Arrival at 13:22:00
 To Station(Solskiego) on line A at 13:22:00 -> Arrival at 13:23:00
 To Station(GRABISZYŃSKA (Cmentarz)) on line A at 13:23:00 -> Arrival at 13:25:00
 To Station(FAT) on line A at 13:25:00 -> Arrival at 13:27:00
 To Station(Aleja Pracy) on line A at 13:27:00 -> Arrival at 13:29:00
 To Station(Inżyniersk

In [29]:
gr.eval_route('17:15:00',['KRZYKI','Reja','RACŁAWICKA','KRZYKI'],True)

(datetime.timedelta(seconds=16260),
 '\n\nFrom KRZYKI  To Station(Orla) on line 7 at 17:18:00 -> Arrival at 17:20:00\n To Station(Jastrzębia) on line 7 at 17:20:00 -> Arrival at 17:21:00\n To Station(Hallera) on line 7 at 17:21:00 -> Arrival at 17:22:00\n To Station(Gajowicka) on line 144 at 17:22:00 -> Arrival at 17:24:00\n To Station(Krucza) on line 144 at 17:24:00 -> Arrival at 17:25:00\n To Station(Żelazna) on line 144 at 17:25:00 -> Arrival at 17:27:00\n To Station(Zaporoska) on line 144 at 17:27:00 -> Arrival at 17:29:00\n To Station(Grabiszyńska) on line 127 at 17:33:00 -> Arrival at 17:34:00\n To Station(Kolejowa) on line 4 at 17:36:00 -> Arrival at 17:37:00\n To Station(pl. Legionów) on line 4 at 17:37:00 -> Arrival at 17:39:00\n To Station(Narodowe Forum Muzyki) on line 4 at 17:39:00 -> Arrival at 17:41:00\n To Station(Zamkowa) on line 4 at 17:41:00 -> Arrival at 17:42:00\n To Station(Świdnicka) on line 4 at 17:42:00 -> Arrival at 17:44:00\n To Station(GALERIA DOMINIKAŃSKA) o

In [30]:
gr.eval_route('17:15:00',['KRZYKI','Reja','RACŁAWICKA','KRZYKI'],False)

(1,
 '\n\n Lines taken : 2\nFrom KRZYKI To Station(Orla) on line 2 at 17:20:00 -> Arrival at 17:22:00\n To Station(Jastrzębia) on line 2 at 17:22:00 -> Arrival at 17:23:00\n To Station(Hallera) on line 2 at 17:23:00 -> Arrival at 17:24:00\n To Station(Sztabowa) on line 2 at 17:24:00 -> Arrival at 17:25:00\n To Station(Rondo) on line 2 at 17:25:00 -> Arrival at 17:26:00\n To Station(Wielka) on line 2 at 17:26:00 -> Arrival at 17:28:00\n To Station(Zaolziańska) on line 2 at 17:28:00 -> Arrival at 17:29:00\n To Station(Arkady (Capitol)) on line 2 at 17:29:00 -> Arrival at 17:32:00\n To Station(DWORZEC GŁÓWNY) on line 2 at 17:32:00 -> Arrival at 17:35:00\n To Station(Wzgórze Partyzantów) on line 2 at 17:35:00 -> Arrival at 17:37:00\n To Station(GALERIA DOMINIKAŃSKA) on line 2 at 17:37:00 -> Arrival at 17:40:00\n To Station(Urząd Wojewódzki (Muzeum Narodowe)) on line 2 at 17:40:00 -> Arrival at 17:43:00\n To Station(Katedra) on line 2 at 17:43:00 -> Arrival at 17:44:00\n To Station(Reja) on

In [31]:
gr.astar('KRZYKI','Inżynierska','17:15:00',time_criterion=True)

(datetime.timedelta(seconds=1260),
 '17:36:00',
 '\n\nFrom KRZYKI  To Station(Orla) on line 7 at 17:18:00 -> Arrival at 17:20:00\n To Station(Jastrzębia) on line 7 at 17:20:00 -> Arrival at 17:21:00\n To Station(Hallera) on line 7 at 17:21:00 -> Arrival at 17:22:00\n To Station(Gajowicka) on line 144 at 17:22:00 -> Arrival at 17:24:00\n To Station(Mielecka) on line 136 at 17:26:00 -> Arrival at 17:27:00\n To Station(Ojca Beyzyma) on line 136 at 17:27:00 -> Arrival at 17:29:00\n To Station(Aleja Pracy) on line 136 at 17:29:00 -> Arrival at 17:30:00\n To Station(Inżynierska) on line 125 at 17:34:00 -> Arrival at 17:36:00\nTime taken : 0:21:00\nArrival at : 17:36:00 ')

In [32]:
gr.astar('pl. Orląt Lwowskich','Inżynierska','14:15:00',time_criterion=False)

(frozenset({125, 127}),
 '23:31:00',
 '\n\n Lines taken : 125,127\nFrom pl. Orląt Lwowskich To Station(Tęczowa) on line 127 at 14:22:00 -> Arrival at 14:23:00\n To Station(Grabiszyńska) on line 127 at 14:23:00 -> Arrival at 14:27:00\n To Station(Zaporoska) on line 127 at 14:27:00 -> Arrival at 14:30:00\n To Station(Krucza) on line 127 at 14:30:00 -> Arrival at 14:31:00\n To Station(Gajowicka) on line 127 at 14:31:00 -> Arrival at 14:33:00\n To Station(Mielecka) on line 127 at 23:11:00 -> Arrival at 23:12:00\n To Station(Ojca Beyzyma) on line 127 at 23:12:00 -> Arrival at 23:13:00\n To Station(Aleja Pracy) on line 127 at 23:13:00 -> Arrival at 23:14:00\n To Station(Inżynierska) on line 125 at 23:30:00 -> Arrival at 23:31:00\nTime taken : 9:16:00\nArrival at : 23:31:00 ')